# A3: A\*, IDS, and Effective Branching Factor

### Zach Goodenow

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this you must include your implementations of these functions, from Assignment 2:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)`
  
Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
   
Define this function so it takes any number of $h$ functions in the list that is the fourth argument.

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

### Compairson Using texttable: usage and example

I found a package online called texttable that lets you write data to an automatically formated text table.  To use it locally you need to run the command "pip install texttable" to install texttable.  The table formating isnt super flexable but I made it work for this example.  I hard coded the values below to get used to using texttable and to show how an example of how you can use it and what the output is.

In [4]:
#Try using texttable
import texttable as tt  

#Make table and add headders
tab = tt.Texttable()
headings = ['Algorithm', 'Depth', 'Nodes', 'EBF']
tab.header(headings)
# tab.set_cols_width([4]*10)
algorithms = ['IDS', 'A*h1', 'A*h2', 'A*h3']

#Example 1
example1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
depths1 = [0,0,0,0]
nodes1 = [0,0,0,0]
ebfs1 = [0.000,0.000,0.000,0.0000]
    
for row in zip(algorithms, depths1, nodes1, ebfs1):
    tab.add_row(row)

exampleTable = tab.draw()
print(exampleTable)

+-----------+-------+-------+-----+
| Algorithm | Depth | Nodes | EBF |
+===========+=======+=======+=====+
| IDS       | 0     | 0     | 0   |
+-----------+-------+-------+-----+
| A*h1      | 0     | 0     | 0   |
+-----------+-------+-------+-----+
| A*h2      | 0     | 0     | 0   |
+-----------+-------+-------+-----+
| A*h3      | 0     | 0     | 0   |
+-----------+-------+-------+-----+


# My Functions

In [2]:
# from A3mysolution import *

## Eight Puzzle Functions
<div class="alert alert-block alert-info">
There are a few changes in these functions from A2 to A3:
<br>
1.	I added a **findIndex_8p function** that returns the index of any piece.  The function findBlank_8p now calls findIndex_8p and passes 0 as the piece.  Later you will see this function is helpful for the heuristic functions, specifically the Manhattan Distance.
<br><br>
2.	The **actionsF_8p function** now returns a tuple where the first element is the string representation of a move and the second is the cost of that move.
<br><br>
3.	The **takeActionF_8p function** requires the tuple format from 2.) as the action.  If this action isn’t in the list returned by actionsF_8p it raises a ValueError.  I thought about accepting just the string representation of the action but I didn't know if that would be helpful or not so I left it out.  

In [63]:
import copy

# Formats 8-puzzle for printing
def printState_8p(state):
    state[state.index(0)] = " "
    print(state[0], state[1], state[2])
    print(state[3], state[4], state[5])
    print(state[6], state[7], state[8])
    state[state.index(" ")] = 0


# Returns index of blank as tuple
def findBlank_8p(state):
    return findIndex_8p(state, 0)


# Returns index of specific piece
def findIndex_8p(state, piece):
    col, row = 0, 0

    # find column
    while col < 3:
        if piece in state[col::3]: break
        col += 1

    # col = 3 if piece wasnt in state
    if col == 3:
        raise ValueError(str(piece) + ' not found in 8-Puzzle!')

    # find row
    for i in state[col::3]:
        if i == piece: break
        row += 1

    return (row, col)


def actionsF_8p(state):
    actions = []
    index = findBlank_8p(state)

    if index[1] > 0:
        actions.append(('left',1))
    if index[1] < 2:
        actions.append(('right',1))
    if index[0] > 0:
        actions.append(('up',1))
    if index[0] < 2:
        actions.append(('down',1))

    return actions


def takeActionF_8p(state, action):
    if action not in actionsF_8p(state):
        raise ValueError(str(action) + ' not valid with blank at index: ' + str(findBlank_8p(state)))

    here = state.index(0) #Index of blank
    there = state.index(0) #Index of spot to move blank
    move = action[0]
    cost = action[1]

    if(move == 'left'):
        there -= 1
    if(move == 'right'):
        there += 1
    if(move == 'up'):
        there -= 3
    if(move == 'down'):
        there += 3

    stateC = copy.copy(state)
    stateC[here], stateC[there] = stateC[there], stateC[here]
    return (stateC, cost)


def goalTestF_8p(state, goal):
    return state == goal


def printPath_8p(startState, goalState, path):
    print('--Start State--')
    printState_8p(startState)
    print('--End State--')
    printState_8p(goalState)

    if isinstance(path, str):
        print('Path not found: ', path)
        return

    print(len(path), ' states for success: ')

    stateNum = 1
    for state in path:
        print('-', stateNum, '-')
        printState_8p(state)
        print()
        stateNum += 1

A little testing of these functions

In [6]:
goalState = [1,2,3,4,0,5,6,7,8]
steps = [goalState]
steps.insert(0, takeActionF_8p(steps[0], ('right', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('down', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('left', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('up', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('left', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('up', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('right', 1))[0])
steps.insert(0, takeActionF_8p(steps[0], ('down', 1))[0])
printPath_8p(steps[0], steps[-1], steps)

--Start State--
2 4 3
1   8
6 5 7
--End State--
1 2 3
4   5
6 7 8
9  states for success: 
- 1 -
2 4 3
1   8
6 5 7

- 2 -
2   3
1 4 8
6 5 7

- 3 -
  2 3
1 4 8
6 5 7

- 4 -
1 2 3
  4 8
6 5 7

- 5 -
1 2 3
4   8
6 5 7

- 6 -
1 2 3
4 5 8
6   7

- 7 -
1 2 3
4 5 8
6 7  

- 8 -
1 2 3
4 5  
6 7 8

- 9 -
1 2 3
4   5
6 7 8



## Heuristic Functions

<div class="alert alert-block alert-info">
1.)	The **mDist function** returns the Manhattan Distance of a piece for a specified start and goal state.  This function is helpful in h2 and h3
<br><br>
2.)	My **h3_8p function** is the summation of the Manhattan Distance of all pieces EXCLUDING the blank piece.  I actually had the idea to use this heuristic in class and later found it in the book.  This heuristic is admissible, and not constant for all states:
    <br>
- An **admissible heuristic** is one that never overestimates the cost of the minimum cost path from a node to the goal node. So, a heuristic is specific to a particular state space, and also to a particular goal state in that state space. It must be **admissible** for all states in that search space.
    <br>
    *	I know **my heuristic is admissible** because to get from any start state to any goal state, each piece needs to move a minimum of its Manhattan distance.  Because each move includes exactly one piece and the blank spot, the Manhattan Distance of each piece is exclusive to that piece alone.  In other words, the Manhattan Distances of all pieces have no overlap.  Thus, my heuristic will never overestimate the cost of the minimum cost path from a node to the goal node.  If I were to include the Manhattan Distance of the blank, this fact would not hold true.  This is because a single move could apply to the Manhattan Distance of a the blank and the piece it switched with. Here is a simple example:
</div>
<div class="alert alert-block alert-success">
Goal: 
$$
\begin{array}{ccc}
0 & 1 & 2\\
3 & 4 & 5\\
6 & 7 & 8
\end{array}
$$
Start:
$$
\begin{array}{ccc}
1 & 0 & 2\\
3 & 4 & 5\\
6 & 7 & 8
\end{array}
$$
<br>
    - H estimate including the blank: 2 –> Blank spot to the left and 1 piece to the right<br>
    - H estimate excluding the blank: 1<br>
    - Switching the blank spot with the 1 piece would result in the goal state.  Thus, the minimum cost path for this example is 1. 
</div>
<div class="alert alert-block alert-info">
- A stronger requirement on a heuristic is that it is **consistent**, sometimes called **monotonic**. A heuristic h is consistent if its value is non-decreasing along a path. Mathematically, a heuristic h is consistent if for every node n of a parent node p, 
$$h(p)≤h(n)+stepcost(p,n)$$
    *	I know that **my heuristic is not consistent** because the Manhattan Distance of any piece decreases every time that piece moves closer to its goal destination.  Obviously, each piece needs to move to its goal spot to satisfy any goal state.  Thus, my heuristic will always decrease at some point along every path from any start state to any goal state.  Hence, my heuristic is not consistent for all states.
    
    
    

In [86]:
# Returns the Manhattan distance that the piece is from its goal position
def mDist(state, goal, piece):
    m = 0
    indexS = findIndex_8p(state, piece)
    indexG = findIndex_8p(goal, piece)
    m += abs(indexS[0] - indexG[0])
    m += abs(indexS[1] - indexG[1])
    return m


# Heuristic Functions
# h(state,goal) = 0, for all states 'state' and all goal states 'goal'
def h1_8p(state, goal):
    return 0


# h(state,goal) = m, where m is the Manhattan distance that the blank is from its goal position
def h2_8p(state, goal):
    return mDist(state, goal, 0)


# h(state,goal) = a, where a is the summation of the Manhattan distance
# that all pieces are from their goal position excluding the blank
def h3_8p(state, goal):
    a = 0
    for piece in range(1,9):
        a += mDist(state, goal, piece)
    return a

A little testing of these functions

In [18]:
goalState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
startState1 = [1, 2, 3, 4, 5, 0, 6, 7, 8] # Actual min. dist. is 1

# Test 1: Move one piece
print('h1: ', h1_8p(startState1, goalState))
print('h2: ', h2_8p(startState1, goalState))
print('h3: ', h3_8p(startState1, goalState))

h1:  0
h2:  1
h3:  1


In [19]:
startState2 = [4, 1, 2, 6, 7, 3, 0, 8, 5] # Actual min. dist. is 8

# Test 2: Move each piece once. 
print('h1: ', h1_8p(startState2, goalState))
print('h2: ', h2_8p(startState2, goalState))
print('h3: ', h3_8p(startState2, goalState))

h1:  0
h2:  2
h3:  8


In [20]:
startState3 = [8, 7, 6, 5, 0, 4, 3, 2, 1]

# Test 3: Inversion of goal
print('h1: ', h1_8p(startState3, goalState))
print('h2: ', h2_8p(startState3, goalState))
print('h3: ', h3_8p(startState3, goalState))

h1:  0
h2:  0
h3:  24


A few examples to test that h3 is not consistent

In [96]:
startState = [1, 5, 0, 4, 7, 2, 6, 8, 3]
goalState = [1, 2, 3, 4, 0, 5, 6, 7, 8]

path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 10)
print('Path length: {} \nstart h3: {}'.format(len(path), h3_8p(startState, goalState)))

Path length: 9 
start h3: 8


In [97]:
for state in path:
    print('State: {} has h value: {}'.format(state, h3_8p(state, goalState)))

State: [1, 5, 0, 4, 7, 2, 6, 8, 3] has h value: 8
State: [1, 5, 2, 4, 7, 0, 6, 8, 3] has h value: 7
State: [1, 5, 2, 4, 7, 3, 6, 8, 0] has h value: 6
State: [1, 5, 2, 4, 7, 3, 6, 0, 8] has h value: 5
State: [1, 5, 2, 4, 0, 3, 6, 7, 8] has h value: 4
State: [1, 0, 2, 4, 5, 3, 6, 7, 8] has h value: 3
State: [1, 2, 0, 4, 5, 3, 6, 7, 8] has h value: 2
State: [1, 2, 3, 4, 5, 0, 6, 7, 8] has h value: 1
State: [1, 2, 3, 4, 0, 5, 6, 7, 8] has h value: 0


## Estimated Effective Branching Factor Functions

Some notes on Effective Branching Factor.
<div class="alert alert-block alert-info">
One way to characterize the quality of a heuristic is the **effective branching factor** $b^∗$. If the total number of nodes generated by A∗ for a particular problem is $N$ and the solution depth is $d$, then $b^∗$ is the branching factor that a uniform tree of depth $d$ would have to have in order to contain $N$ nodes. Thus,
</div>
<div class="alert alert-block alert-success">
$$
N = 1 + b^∗ + (b^∗)^2 + ··· + (b^∗)^d
$$

A faster way to calculate the above quantity is

$$
\frac{1-b^{d+1}}{1-b}
$$
</div>
<div class="alert alert-block alert-info">
For example, if A∗ finds a solution at depth 5 using 52 nodes, then the effective branching
factor is 1.92. That is, if there was a tree 5 layers deep where each node had 1.92 children, then the tree would contain 52 nodes in total.
<br><br>
A well designed heuristic would have a value of $b^∗$ close to 1, allowing fairly large problems to be solved at reasonable computational cost.

Some notes on my functions.
<div class="alert alert-block alert-info">
The **ebf function** returns an estimated effective branching factor $b^∗$ given the number of nodes expanded during a search and the depth of the search.  The precision of the number of nodes that the returned $b^∗$ will estimate can also be applied but will default to 0.01 if not specified.  Precision is important here because $b^∗$ is being searched for using a binary search method.
<br><br>
Originally, I thought that ebf couldnt be estimated when number of nodes was less than 1 or depth was less than 0 so I was raising a ValueError for those cases.  Then I saw an example in the "Local Search in Discrete State and Action Spaces" notes that ebf(0, 0) was simply 0 and ebf(1, 1) was 0.0078125.  So I commented these ValueErrors out and simply returned 0 when nNodes was less than 1 or depth was less than 0.  
<br><br>
**CASE:** depth = 1.  Mathematically, when depth = 1, $$N = 1 + b^*$$ Hence, $$b^* = N - 1$$  
I had a case in my function that would return nNodes-1 when depth = 1 but I commented this case out because there are examples where depth = 1 and the returned value is not nNodes-1.  Based off of these examples, I assume that Chuck's implementation doesnt have a hard-coded case for depth = 1.
<br><br>
**CASE:** depth = 0.  Intuitively, when depth = 0, that means that the solution was found before any nodes were expanded.  Since the solution was found instantaneously, the effective branching factor should be 1.0.  But for this case, the ebf is not very applicable.
<br><br>
**CASE:** nNodes = 1.  Mathematically, when N = 1, $$1 = 1 + b^* + (b^*)^2 + ...$$ Hence, $b^*$ must equal 0.  However, there are a few examples from the notes where nNodes = 1 and the returned value is between 0 and 1.  Again, I commented this case out and let my function do the work
<br><br>
**CASE:** nNodes less than 1 or depth less than 0.  The ebf measurement doesn't really apply to this case either but examples in the notes (ebf(0,0)) led me to return 0 rather than raise a ValueError.
<br><br>
The **ebfGuess function** returns the number of nodes expanded during a search given an estimated $b^∗$ and depth.  This is helpful because ebf needs this equation very frequently.  
<br>
**SOMETHING COOL:** I was able to bring my ebfGuess function down to a single line by manipulating lists, range, and summation.  THIS IS WHY I LOVE PYTHON!

In [65]:
# nNodes = number of nodes expanded during a search
# depth = depth of solution for a search
# precision = precision on the number of nodes that the returned b* will estimate
# returns esimated b*
def ebf(nNodes, depth, precision=0.01):
    if nNodes < 1 or depth < 0:
        return 0
    if depth == 0:
        return 1.0
#     if nNodes == 1:
#         return 0.0
#     if depth == 1: # N = 1 + b* => b* = N - 1
#         return nNodes-1
#     if nNodes < 1:
#         raise ValueError('nNodes cant be less than 1 in ebf. Input: ' + str(nNodes))
#     if depth < 0:
#         raise ValueError('depth cant be less than 0 in ebf. Input: ' + str(depth))


    # Find range.  Try 10 first and go up by 10s
    low = 1 if ebfGuess(1, depth) <= nNodes else 0
    high = 10
    while nNodes > ebfGuess(high, depth):
        low = high
        high += 10

    # print('range is', low, '-', high)
    # Binary search for answer between range
    while True:
        guess = (high + low) / 2
        result = ebfGuess(guess, depth)
        # if result-precision <= nNodes <= result+precision:
        if abs(nNodes - result) < precision:
            return guess
        # print('trying a b value of', guess, 'with range', low, '-', high)
        if result > nNodes:
            high = guess
        else:
            low = guess


def ebfGuess(b, depth):
    return sum([b**power for power in range(depth+1)])
#     result = 0
#     for power in range(depth+1):
#         result += b**power
#     return result

First, some example output for the ebf function.

In [38]:
ebf(10, 3) # 1.661376953125

1.661376953125

The smallest argument values should be a depth of 0, and 1 node.

In [39]:
ebf(1, 0) # 1.0

1.0

In [40]:
ebf(1, 1) # 0.0078125

0.009765625

In [41]:
ebf(1, 100)

0.009765625

In [42]:
ebf(2, 1) # 1.0078125

1.0087890625

In [43]:
ebf(3, 1)

2.001953125

In [44]:
ebf(10, 1)

8.998046875

In [45]:
ebf(4, 0)

1.0

In [46]:
# Check that precision is working correctly
ebf(2, 1, precision=0.000001) # 1.0000009536743164

1.000000536441803

In [47]:
ebf(100, 12, 0.01) # 1.3034343719482422

1.3034286499023438

In [48]:
ebf(200000, 5) # 11.275596931956898

11.275596916675568

In [49]:
ebf(200000, 50) # 1.2348192492705223

1.2348192499484867

## Node Class

<div class="alert alert-block alert-info">
The **Node class** is used to make node objects.  This is extremely useful when using heuristics and informed search.  This is how the aStarSearch function will keep track of f, g, and h. I found this code in the notes "08 Python Classes" from 440 lecture

In [66]:
# Node class used for keeping track of f, g, & h for each state
class Node:
    # Constructor
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h

    # String representation of a node
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

## Global Variables

<div class="alert alert-block alert-info">
The global **nodesExpanded** is used to count the number of nodes/states expanded by each search algorithm.  I set this variable to 0 each time a new search is started to make sure I dont count nodes from a previous search.

In [81]:
# Global to count nodes expanded
nodesExpanded = 0

## Iterative Deepening Search Functions

<div class="alert alert-block alert-info">
A few changes from A2 to A3.
<br>
1. The **nodesExpanded** global is used to count the nodes expanded by each call to iterativeDeepeningSearch
<br>
2. The **takeActionF function** is now returning a tuple with child state and step cost.

In [67]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global nodesExpanded
    nodesExpanded = 0
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result.insert(0, startState)
            return result
    return 'cutoff'


def depthLimitedSearch(startState, goalState, actionsF, takeActionF, depthLimit):
    return recursiveDLS(startState, goalState, actionsF, takeActionF, depthLimit)


def recursiveDLS(startState, goalState, actionsF, takeActionF, depthLimit):
    global nodesExpanded
    if startState == goalState:
        return []
    elif depthLimit == 0:
        return 'cutoff'
    else:
        cuttoffOccured = False
        for action in actionsF(startState):
            child,_ = takeActionF(startState, action)
            nodesExpanded += 1
            result = recursiveDLS(child, goalState, actionsF, takeActionF, depthLimit-1)
            if result == 'cutoff':
                cuttoffOccured = True
            elif result != 'failure':
                result.insert(0, child)
                return result
        if cuttoffOccured:
            return 'cutoff'
        else:
            return 'failure'

## A Star Search Functions

<div class="alert alert-block alert-info">
The **aStarSearch function** uses the start state (**startState**), valid actions function (**actionsF**), action application function (**takeActionF**), goal test function (**goalTestF**), and heuristic function passed (**hF**), to make a new node.  It then uses this node to call the **aStarSearchHelper function** to recursively best-search.  This is helpful because the user of the aStarSearch function doesnt need to know how to use the node class at all.  It also keeps track of the f, g, and h values to find the next best child node to explore.  
<br>
A majority of this code was taken from the notes "07 Informed Search" from 440 lecture but modified slightly.  The code in the 07 notes actually has a error when trying to search for a node that does not exist.  However, for this assignment, we were told that we may assume that the goal is reachable. Failure will not happen for any of the tests run when grading your solution.  Even so, I use a try catch for the call to aStarSearchHelper to catch "RecursionError: maximum recursion depth exceeded" and return failure.
<br><br>
**Using lambda to call aStarSearch**
<br>
An interesting piece of these functions is that aStarSearchHelper only passes one argument to goalTestF and hF when they are called.  Originally, I thought that goalState was forgotten as an argument in the definition of aStarSeach.  But then I saw the example provided that used lambda to pass the goal state when calling aStarSearch.  I was very unfamilar with using lambda to do anything other than sort lists.  After reading posts on piazza and some google research, I figured out how python was interprateing this.  Here is some text about lambda from 'https://www.python-course.eu/lambda.php':
- "The lambda operator or lambda function is a way to create small anonymous functions, i.e. functions without a name. These functions are throw-away functions, i.e. they are just needed where they have been created."
- "The general syntax of a lambda function is quite simple: `lambda argument_list: expression`.
The argument list consists of a comma separated list of arguments and the expression is an arithmetic expression using these arguments. You can assign the function to a variable to give it a name."

Basically, a new function is beind defined when calling lambda in this case.  This function is identical to the goal test function or heuristic function already defined, but when it is called in the aStar functions, it passes the goal state provided as the second arguement.  
<br>
TL;DR: 
- Problem: 1.) aStar functions only pass the current state to goalTestF and hF.  2.) My goalTestF's and hF's require two arguments; state, goal.
- Solution: Use lambda to define a new function with a hardcoded goal argument for goalTestF and hF

In [68]:
# Recursive Best-First Search implementation of the A* algorithm given in class
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
# Recursive Iterative Deepening form of A*, where depth is replaced by f(n)
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global nodesExpanded
    nodesExpanded = 0
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    try: 
        result = aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))
    except RuntimeError as re: # If  RecursionError occurs just return failure
        result = ("failure", float('inf'))
    return result


def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global nodesExpanded
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    # Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState, stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
        nodesExpanded += 1
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)

        # next lowest f value
        altf = children[1].f if len(children) > 1 else float('inf')

        # expand best child, reassign its f value to be returned value
        result, bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF, hF, min(fmax,altf))
        if result is not "failure":
            result.insert(0, parentNode.state)
            return (result, bestChild.f)

Here is a simple example using our usual simple graph search.

In [53]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [54]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [55]:
takeActionF_simple('a', actions[0])

('b', 1)

In [56]:
goalTestF_simple('a', 'a')

True

In [57]:
h_simple('a', 'z')

1

In [69]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [70]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Search for a goal that doesnt exist

In [71]:
aStarSearch('a', actionsF_simple, takeActionF_simple,
                lambda s: goalTestF_simple(s, 'q'),
                lambda s: h_simple(s, 'q'))

('failure', inf)

Example from A2 that was already solved.  All 4 versions of my search functions should be able to solve this example.

In [80]:
startState = [1, 5, 0, 4, 7, 2, 6, 8, 3]
goalState = [1, 2, 3, 4, 0, 5, 6, 7, 8]

iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 10)

[[1, 5, 0, 4, 7, 2, 6, 8, 3],
 [1, 5, 2, 4, 7, 0, 6, 8, 3],
 [1, 5, 2, 4, 7, 3, 6, 8, 0],
 [1, 5, 2, 4, 7, 3, 6, 0, 8],
 [1, 5, 2, 4, 0, 3, 6, 7, 8],
 [1, 0, 2, 4, 5, 3, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 6, 7, 8],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [87]:
aStarSearch(startState, actionsF_8p, takeActionF_8p,
                lambda s: goalTestF_8p(s, goalState),
                lambda s: h1_8p(s, goalState))

([[1, 5, 0, 4, 7, 2, 6, 8, 3],
  [1, 5, 2, 4, 7, 0, 6, 8, 3],
  [1, 5, 2, 4, 7, 3, 6, 8, 0],
  [1, 5, 2, 4, 7, 3, 6, 0, 8],
  [1, 5, 2, 4, 0, 3, 6, 7, 8],
  [1, 0, 2, 4, 5, 3, 6, 7, 8],
  [1, 2, 0, 4, 5, 3, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 0, 5, 6, 7, 8]],
 8)

In [88]:
aStarSearch(startState, actionsF_8p, takeActionF_8p,
                lambda s: goalTestF_8p(s, goalState),
                lambda s: h2_8p(s, goalState))

([[1, 5, 0, 4, 7, 2, 6, 8, 3],
  [1, 5, 2, 4, 7, 0, 6, 8, 3],
  [1, 5, 2, 4, 7, 3, 6, 8, 0],
  [1, 5, 2, 4, 7, 3, 6, 0, 8],
  [1, 5, 2, 4, 0, 3, 6, 7, 8],
  [1, 0, 2, 4, 5, 3, 6, 7, 8],
  [1, 2, 0, 4, 5, 3, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 0, 5, 6, 7, 8]],
 8)

In [91]:
aStarSearch(startState, actionsF_8p, takeActionF_8p,
                lambda s: goalTestF_8p(s, goalState),
                lambda s: h3_8p(s, goalState))

([[1, 5, 0, 4, 7, 2, 6, 8, 3],
  [1, 5, 2, 4, 7, 0, 6, 8, 3],
  [1, 5, 2, 4, 7, 3, 6, 8, 0],
  [1, 5, 2, 4, 7, 3, 6, 0, 8],
  [1, 5, 2, 4, 0, 3, 6, 7, 8],
  [1, 0, 2, 4, 5, 3, 6, 7, 8],
  [1, 2, 0, 4, 5, 3, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 0, 5, 6, 7, 8]],
 8)

The following example is proof that my heuristic greatly improves the search optimality of $A*$.  I tried to design a start and goal state that would require the most states to get from start to goal.  In this case, the start state is simply an inverse of the goal state.  I came to this example because it maximized the h value of my heuristic function.  I actually brought up this example in the heuristic section of this notebook where I show the output of my heuristic compaired to the other two.  When I tried to apply this example to IDS, $A*$ with h1, and even $A*$ with h2, they were unable to generate a solution in a reasonable time.  
<div class="alert alert-block alert-success">
Goal: 
$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$
Start:
$$
\begin{array}{ccc}
8 & 7 & 6\\
5 & 0 & 4\\
3 & 2 & 1
\end{array}
$$
<br>

In [76]:
goalState = [1,2,3,4,0,5,6,7,8]
hardStart = [8,7,6,5,0,4,3,2,1]
hardPath = aStarSearch(hardStart, actionsF_8p, takeActionF_8p,
                lambda s: goalTestF_8p(s, goalState),
                lambda s: h3_8p(s, goalState))
print('Best f value in path: ', hardPath[1])
print('Path length: ', len(hardPath[0]))
print('Path: ', hardPath[0])

Best f value in path:  30
Path length:  31
Path:  [[8, 7, 6, 5, 0, 4, 3, 2, 1], [8, 7, 6, 5, 2, 4, 3, 0, 1], [8, 7, 6, 5, 2, 4, 3, 1, 0], [8, 7, 6, 5, 2, 0, 3, 1, 4], [8, 7, 0, 5, 2, 6, 3, 1, 4], [8, 0, 7, 5, 2, 6, 3, 1, 4], [0, 8, 7, 5, 2, 6, 3, 1, 4], [5, 8, 7, 0, 2, 6, 3, 1, 4], [5, 8, 7, 3, 2, 6, 0, 1, 4], [5, 8, 7, 3, 2, 6, 1, 0, 4], [5, 8, 7, 3, 2, 6, 1, 4, 0], [5, 8, 7, 3, 2, 0, 1, 4, 6], [5, 8, 0, 3, 2, 7, 1, 4, 6], [5, 0, 8, 3, 2, 7, 1, 4, 6], [0, 5, 8, 3, 2, 7, 1, 4, 6], [3, 5, 8, 0, 2, 7, 1, 4, 6], [3, 5, 8, 1, 2, 7, 0, 4, 6], [3, 5, 8, 1, 2, 7, 4, 0, 6], [3, 5, 8, 1, 2, 7, 4, 6, 0], [3, 5, 8, 1, 2, 0, 4, 6, 7], [3, 5, 0, 1, 2, 8, 4, 6, 7], [3, 0, 5, 1, 2, 8, 4, 6, 7], [0, 3, 5, 1, 2, 8, 4, 6, 7], [1, 3, 5, 0, 2, 8, 4, 6, 7], [1, 3, 5, 4, 2, 8, 0, 6, 7], [1, 3, 5, 4, 2, 8, 6, 0, 7], [1, 3, 5, 4, 2, 8, 6, 7, 0], [1, 3, 5, 4, 2, 0, 6, 7, 8], [1, 3, 0, 4, 2, 5, 6, 7, 8], [1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]]


# Comparison using texttable

In [72]:
import texttable as tt 

def runExperiment(goalState1, goalState2, goalState3, hList): 
    #Make table and add headders
    tab = tt.Texttable()
    headings = ['Algorithm', 'Depth', 'Nodes', 'EBF', 'Depth', 'Nodes', 'EBF', 'Depth', 'Nodes', 'EBF']
    tab.header(headings)
    tab.set_cols_width([7]*10)
    algs = ['IDS']
    for num in range(len(hList)):
        algs.append('A*h' + str(num+1))
    
    startState = [1,2,3,4,0,5,6,7,8]

    depths1, nodes1, ebfs1 = experiment(startState, goalState1, hList)
    depths2, nodes2, ebfs2 = experiment(startState, goalState2, hList)
    depths3, nodes3, ebfs3 = experiment(startState, goalState3, hList)

    print('State:    |', goalState1, '|', goalState2, '|', goalState3, '|')
    for row in zip(algs, depths1, nodes1, ebfs1, depths2, nodes2, ebfs2, depths3, nodes3, ebfs3):
        tab.add_row(row)
    exampleTable = tab.draw()
    print(exampleTable)


# Helper function for runExperiment. Called once per example and returns lists of data
def experiment(startState, goalState, hList):
    global nodesExpanded
    depths = []
    nodes = []
    ebfs = []

    result = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 12)
    depths.append(len(result)-1)
    nodes.append(nodesExpanded)
    est = ebf(nodes[-1], depths[-1]) if nodes[-1] > 0 else 0
    ebfs.append(est)

    for hfxn in hList:
        result = aStarSearch(startState, actionsF_8p, takeActionF_8p,
                             lambda s: goalTestF_8p(s, goalState),
                             lambda s: hfxn(s, goalState))
        depths.append(len(result[0])-1)
        nodes.append(nodesExpanded)
        est = ebf(nodes[-1], depths[-1]) if nodes[-1] > 0 else 0
        ebfs.append(est)

    return depths, nodes, ebfs

# Usage...
goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]
runExperiment(goalState1, goalState2, goalState3, [h1_8p, h2_8p, h3_8p])

State:    | [1, 2, 3, 4, 0, 5, 6, 7, 8] | [1, 2, 3, 4, 5, 8, 6, 0, 7] | [1, 0, 3, 4, 5, 8, 2, 6, 7] |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Algorit |  Depth  |  Nodes  |   EBF   |  Depth  |  Nodes  |   EBF   |  Depth  |  Nodes  |   EBF   |
|   hm    |         |         |         |         |         |         |         |         |         |
+=========+=========+=========+=========+=========+=========+=========+=========+=========+=========+
| IDS     | 0       | 0       | 0       | 3       | 43      | 3.086   | 11      | 225850  | 2.954   |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| A*h1    | 0       | 0       | 0       | 3       | 116     | 4.488   | 11      | 643246  | 3.263   |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| A*h2    | 0       | 0       | 0       | 3       | 51      | 3.297   | 11      | 

The output of this tests shows that the accuracy of the heuristic used in $A*$ has heavy influence on the optimality of the search.  Specifically, $A*$ is less optimal than IDS when the heuristic is set to always return 0.  It also shows that my heuristic is much better compared to the other two in terms of optimality.  
<br>
Another interesting statement brought up by these results are that consistent heuristics are not always more optimal.  When I was reading about heuristics I assumed that a consistent heuristic would always be more optimal, but this is definitely not the case.

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [88]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            